In [488]:
import os
import pandas as pd
import json
import folium
import seaborn as sns
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import csv
%matplotlib inline 
from langdetect import detect, detect_langs
import unicodedata

def has_letters(s):
    return any([unicodedata.category(c) in ['Ll','Lu','Lo'] for c in s])

In [501]:
with open("twitter-swisscom/twex.tsv", 'r') as f:
    read_data = f.read().replace("\\\n"," ").split("\n")
read_data=list(map(lambda x: x.split("\t"),read_data))
read_data=read_data[0:-1]
tdata=pd.DataFrame(read_data)

OSError: [Errno 22] Invalid argument

In [495]:
schema=pd.read_csv("twitter-swisscom/schema.txt",sep="\s+",quoting=csv.QUOTE_NONE,header=None)
schema.drop([0,3,4,5],axis=1,inplace=True)
tdata.columns=schema[1]

In [ ]:
lang_codes=pd.read_csv("twitter-swisscom/language-codes_csv.csv")
lang_codes.set_index("alpha2",inplace=True)

In [ ]:
import re
tdata["text_stripped"]=tdata.text.map(lambda x: re.sub(r'https\S+', '', x))
tdata["text_stripped"]=tdata["text_stripped"].map(lambda x: re.sub(r'#\S+', '', x))
tdata["text_stripped"]=tdata.text_stripped.map(lambda x: re.sub(r'@\S+', '', x))

In [ ]:
tdata["lang"]=tdata.text_stripped.map(lambda x: detect(x) if has_letters(x) else np.nan)
tdata["language"]=tdata.lang.map(lambda x: lang_codes.English[x] if not pd.isnull(x) else np.nan)
tdata["language"].replace({np.nan:"Undetected"},inplace=True)

In [ ]:
pd.to_pickle(tdata,"tdata_lang.p")

In [ ]:
counts=tdata.groupby("language").id.count().sort_values(ascending=False)
counts=counts.reset_index()
counts.columns=["language","count"]
pd.to_pickle(counts,"lang_counts.p")

In [ ]:
ax = sns.factorplot(size=12,kind = 'bar', y="language", x="count", data=counts, orient='h')
plt.xticks(rotation=90,fontsize=15)
plt.show(ax)

In [ ]:
#pd.set_option("display.max_rows",500)
#tdata.loc[tdata.language==""].text_stripped[0:500]